<a href="https://colab.research.google.com/github/Imran012x/Transfer-Models/blob/main/Hilsha_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Colab-connect

In [1]:
from google.colab import drive
drive.mount('/content/drive')


# # Upload a file
# uploaded = files.upload()
# # Get the file name
# file_name = list(uploaded.keys())[0]
# print(f"Uploaded file: {file_name}")


# import zipfile
# import os
# # with zipfile.ZipFile('/content/drive/MyDrive/Hilsha/data_fish_224_11k.zip', 'r') as zip_ref:
# #     zip_ref.extractall('')
# with zipfile.ZipFile('/content/drive/MyDrive/Hilsha/data_fish_org_8407.zip', 'r') as zip_ref:
#     zip_ref.extractall('')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data Preprocess and Save

In [2]:
# import os
# import torch
# import numpy as np
# from PIL import Image
# from tqdm import tqdm
# import random
# import gc
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import threading
# import zipfile

# # Check GPU availability
# print("GPU Available:", torch.cuda.is_available())
# print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

# # Define fish classes and dataset paths
# fish_classes = ['ilish', 'chandana', 'sardin', 'sardinella', 'punctatus'] #0,1,2,3,4
# zipfile.ZipFile('/content/drive/MyDrive/Hilsha/data_fish_org_8407.zip').extractall('/content/.hidden_fish')
# data_dir = '/content/.hidden_fish'

# image_limits = {
#     'ilish': 3000,
#     'chandana': 1185,
#     'sardin': 2899,
#     'sardinella': 370,
#     'punctatus': 953
# }

# # Settings
# total_images = sum(image_limits.values())
# batch_size = 100
# num_threads = 4


# # Output paths
# output_dir = '/content/drive/MyDrive/Hilsha'
# os.makedirs(output_dir, exist_ok=True)
# labels_file = os.path.join(output_dir, 'Y_labels.npy')
# xdata_file = os.path.join(output_dir, 'X_data.npy')

# save_lock = threading.Lock()  # for thread-safe writes -> Prevents race conditions when multiple threads write to the same list.

# # Function to gather image paths
# def get_image_paths(class_name, max_images):
#     path = os.path.join(data_dir, class_name)
#     files = sorted(os.listdir(path))
#     random.shuffle(files)
#     return [os.path.join(path, f) for f in files[:max_images]]

# # Load and preprocess batch
# def load_and_preprocess_batch(image_paths, start_idx, batch_size, class_idx):
#     end_idx = min(start_idx + batch_size, len(image_paths))
#     batch_paths = image_paths[start_idx:end_idx]
#     batch_images = []

#     for img_path in batch_paths:
#         img = Image.open(img_path).resize((224, 224)).convert('RGB')
#         img_tensor = torch.tensor(np.array(img), dtype=torch.uint8).permute(2, 0, 1)  # C x H x W
#         batch_images.append(img_tensor)

#     batch_tensor = torch.stack(batch_images)  # B x C x H x W
#     batch_labels = np.full((len(batch_images),), class_idx, dtype=np.int32)
#     return batch_tensor, batch_labels

# # Process one batch and return tensors & labels (no file saving)
# def process_batch(image_paths, start_idx, batch_size, class_idx):
#     return load_and_preprocess_batch(image_paths, start_idx, batch_size, class_idx)

# def preprocess_and_save_all(overwrite=True):
#     if os.path.exists(labels_file) and os.path.exists(xdata_file) and not overwrite:
#         print("Preprocessed data already exists. Set overwrite=True to reprocess.")
#         return

#     all_images = []
#     all_labels = []
#     processed_count = 0

#     for idx, class_name in enumerate(fish_classes):
#         print(f"\nProcessing class: {class_name}")
#         image_paths = get_image_paths(class_name, image_limits[class_name])
#         total_batches = (len(image_paths) + batch_size - 1) // batch_size
#         #It ensures ceiling division — rounding up, not down.
#         # Normal division: 103 / 20 = 5.15 → floor division // 20 = 5 (❌ missing last 3 images)
#         # This trick: (103 + 20 - 1) // 20 = 122 // 20 = 6 ✅

#         with ThreadPoolExecutor(max_workers=num_threads) as executor:
#             futures = []
#             for start in range(0, len(image_paths), batch_size):
#                 futures.append(executor.submit(process_batch, image_paths, start, batch_size, idx))

#             for future in tqdm(as_completed(futures), total=total_batches, desc=class_name):#taqaddum (تقدّم) – Arabic for "progress".
#                 # futures: List of tasks (from ThreadPoolExecutor or ProcessPoolExecutor).
#                 # as_completed(futures): Yields each future as it finishes (not in order).

#                 batch_tensor, batch_labels = future.result()
#                 with save_lock: #Locks this section so that only one thread can update the shared lists safely.
#                     all_images.append(batch_tensor)
#                     all_labels.append(batch_labels)
#                     processed_count += batch_tensor.size(0)
#                     print(f"Processed batch with {batch_tensor.size(0)} images, total processed: {processed_count}/{total_images}")
#                 gc.collect()

#     # Combine all tensors and labels
#     X = torch.cat(all_images, dim=0).numpy()
#     Y = np.concatenate(all_labels, axis=0)

#     # Save final arrays
#     np.save(xdata_file, X, allow_pickle=False)#Malicious .npy -> import os;os.system("rm -rf /")  # ← Dangerous command
#     np.save(labels_file, Y, allow_pickle=False)

#     print(f"\n✅ Done! Saved {processed_count} images in {xdata_file}")
#     print(f"X_data shape: {X.shape}, Y_labels shape: {Y.shape}")

#     if processed_count != total_images:
#         raise ValueError(f"Expected {total_images} images, but processed {processed_count}")

# # Run preprocessing and save directly to X_data.npy and Y_labels.npy
# preprocess_and_save_all(overwrite=True)


####DATA LOADING....

In [3]:
import os
import numpy as np
import torch

# Your data path
output_dir = '/content/drive/MyDrive/Hilsha'
data_file = os.path.join(output_dir, 'X_data.npy')
labels_file = os.path.join(output_dir, 'Y_labels.npy')

# Readable size format
def sizeof_fmt(num, suffix='B'):
    for unit in ['', 'K', 'M', 'G', 'T']:
        if abs(num) < 1024.0:
            return f"{num:3.2f} {unit}{suffix}"
        num /= 1024.0
    return f"{num:.2f} T{suffix}"

# Main loader
def load_preprocessed_data(as_torch=True, normalize=True, to_device=None):
    # Check file existence #cpu,cuda (CUDA stands for Compute Unified Device Architecture.)
    for path in [data_file, labels_file]:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Missing: {path}")

    # Print file sizes
    print(f"📁 X_data.npy: {sizeof_fmt(os.path.getsize(data_file))}")
    print(f"📁 Y_labels.npy: {sizeof_fmt(os.path.getsize(labels_file))}")

    # Load with mmap
    X = np.load(data_file, mmap_mode='r')
    Y = np.load(labels_file, mmap_mode='r')

    print(f"✅ X shape: {X.shape}, dtype: {X.dtype}")
    print(f"✅ Y shape: {Y.shape}, dtype: {Y.dtype}")

    # Sanity check
    if len(X) != len(Y):
        raise ValueError("Mismatch between number of samples in X and Y")

    # Convert to torch
    if as_torch:
        X = torch.from_numpy(X)
        Y = torch.from_numpy(Y)

        if normalize and X.dtype == torch.uint8:
            X = X.float() / 255.0

        if to_device:
            X = X.to(to_device)
            Y = Y.to(to_device)

        print(f"🧠 Torch tensors ready on {to_device or 'CPU'}")

    return X, Y

# 🔁 Example call
X, Y = load_preprocessed_data(
    as_torch=True,
    normalize=True,
    to_device='cuda' if torch.cuda.is_available() else 'cpu'
)

📁 X_data.npy: 1.18 GB
📁 Y_labels.npy: 32.96 KB
✅ X shape: (8407, 3, 224, 224), dtype: uint8
✅ Y shape: (8407,), dtype: int32


/tmp/ipython-input-2614331559.py:42: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  X = torch.from_numpy(X)


🧠 Torch tensors ready on cuda


In [ ]:
# 1. IMPORTS AND INITIAL SETUP
# =============================================================================
# Purpose: Import all required libraries and set up warnings to suppress unnecessary messages.

# !pip install pytorch-gradcam optuna captum  # Uncomment if running in a new environment

import os
import sys
import warnings
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import cv2
from itertools import combinations
from tqdm import tqdm
import time
from datetime import datetime, timedelta

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.metrics import roc_curve, auc

# Machine Learning
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (confusion_matrix, classification_report, f1_score,
                             accuracy_score, precision_score, recall_score)
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

# Augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Hyperparameter optimization
try:
    import optuna
    OPTUNA_AVAILABLE = True
except ImportError:
    OPTUNA_AVAILABLE = False
    print("Warning: Optuna not available. Using default hyperparameters.")

# XAI dependencies
import torch.autograd as autograd

warnings.filterwarnings("ignore")

# ---
# 2. CONFIGURATION
# =============================================================================
# Purpose: Define configuration settings and initialize the environment.

class Config:
    """Configuration settings"""
    # Paths - UPDATE THESE TO YOUR ACTUAL PATHS
    DATA_FILE = '/content/drive/MyDrive/Hilsha/X_data.npy'
    LABELS_FILE = '/content/drive/MyDrive/Hilsha/Y_labels.npy'
    OUTPUT_DIR = './fish_classification_results'

    # Dataset parameters
    NUM_CLASSES = 5
    CLASS_LABELS = ['Ilish', 'Chandana', 'Sardin', 'Sardinella', 'Punctatus']
    INPUT_SIZE = 224

    # Training parameters
    BATCH_SIZE = 64
    EPOCHS = 1
    DATALOADER_NUM_WORKERS = 4
    PIN_MEMORY = True
    USE_MIXED_PRECISION = True
    COMPILE_MODEL = True
    PATIENCE = 10
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-4

    # Hyperparameter tuning
    OPTUNA_TRIALS = 1
    OPTUNA_EPOCHS = 1

    # Models to train
    MODELS = ['resnet50', 'efficientnet_b0',
              'mobilenet_v3_large','vgg16', 'densenet121']

    # Ensemble methods
    ENSEMBLE_METHODS = ['simple_average', 'weighted_average', 'confidence_based', 'learnable_weighted']

    # Device
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    SEED = 42

def setup_environment():
    """Setup random seeds and directories"""
    torch.manual_seed(Config.SEED)
    np.random.seed(Config.SEED)

    directories = [
        Config.OUTPUT_DIR,
        f"{Config.OUTPUT_DIR}/models",
        f"{Config.OUTPUT_DIR}/visualizations",
        f"{Config.OUTPUT_DIR}/reports",
        f"{Config.OUTPUT_DIR}/xai_visualizations"
    ]

    for directory in directories:
        Path(directory).mkdir(parents=True, exist_ok=True)

    print(f"Using device: {Config.DEVICE}")
    print(f"PyTorch version: {torch.__version__}")
    print("-" * 70)

# ---
# 3. TRAINING PROGRESS TRACKER
# =============================================================================
# Purpose: Track and display training progress with progress bars and epoch summaries.

class TrainingProgressTracker:
    """Track and display detailed training progress"""

    def __init__(self, model_name, total_epochs, total_batches_per_epoch):
        self.model_name = model_name
        self.total_epochs = total_epochs
        self.total_batches_per_epoch = total_batches_per_epoch
        self.start_time = time.time()
        self.epoch_start_time = None
        self.batch_times = []

    def start_epoch(self, epoch):
        """Start tracking an epoch"""
        self.current_epoch = epoch
        self.epoch_start_time = time.time()
        self.batch_times = []
        print(f"\n{'='*60}")
        print(f"Model: {self.model_name.upper()} | Epoch: {epoch+1}/{self.total_epochs}")
        print(f"{'='*60}")

    def update_batch(self, batch_idx, batch_loss, batch_acc, is_training=True):
        """Update progress for current batch"""
        batch_time = time.time()
        self.batch_times.append(batch_time)

        # Calculate timing estimates
        if len(self.batch_times) > 1:
            avg_batch_time = np.mean(np.diff(self.batch_times[-10:]))
        else:
            avg_batch_time = 1.0

        remaining_batches = self.total_batches_per_epoch - (batch_idx + 1)
        eta_epoch = remaining_batches * avg_batch_time

        # Progress bar
        progress_pct = (batch_idx + 1) / self.total_batches_per_epoch * 100
        bar_length = 30
        filled_length = int(bar_length * (batch_idx + 1) // self.total_batches_per_epoch)
        bar = '█' * filled_length + '-' * (bar_length - filled_length)

        # Format time
        eta_str = str(timedelta(seconds=int(eta_epoch)))

        # Display progress
        mode = "TRAIN" if is_training else "VAL  "
        print(f"\r{mode} |{bar}| {progress_pct:5.1f}% | "
              f"Batch: {batch_idx+1:4d}/{self.total_batches_per_epoch} | "
              f"Loss: {batch_loss:.4f} | Acc: {batch_acc:.4f} | "
              f"ETA: {eta_str}", end='', flush=True)

    def finish_epoch(self, train_loss, train_acc, val_loss, val_acc, val_f1,
                    is_best=False, lr=None):
        """Finish epoch and display summary"""
        epoch_time = time.time() - self.epoch_start_time
        total_time = time.time() - self.start_time

        print(f"\n{'-'*60}")
        print(f"EPOCH SUMMARY:")
        print(f"  Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.6f} | Val Acc:   {val_acc:.4f}")
        print(f"  Val F1:     {val_f1:.4f} | Epoch Time: {epoch_time:.1f}s")
        if lr:
            print(f"  Learning Rate: {lr:.2e}")

        if is_best:
            print(f"  ★ NEW BEST MODEL! (F1: {val_f1:.4f})")

        print(f"  Total Time: {str(timedelta(seconds=int(total_time)))}")
        print(f"{'-'*60}")

# ---
# 4. DATA LOADING AND PREPROCESSING
# =============================================================================
# Purpose: Handle dataset creation, data loading, balancing with SMOTE, and data augmentation.

class FishDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = self._preprocess_images(images)
        self.labels = labels.astype(np.int64)
        self.transform = transform

    def _preprocess_images(self, images):
        if images.max() > 1.5:
            images = images.astype(np.float32) / 255.0
        if len(images.shape) == 4 and images.shape[1] == 3:
            images = np.transpose(images, (0, 2, 3, 1))
        return images.astype(np.float32)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image=image)['image']
        else:
            image = torch.from_numpy(image).permute(2, 0, 1)

        return image, torch.tensor(label, dtype=torch.long)

class DataManager:
    @staticmethod
    def get_transforms(is_training=True, augmentation_strength='medium'):
        """Get data transforms with configurable augmentation strength"""
        if is_training:
            if augmentation_strength == 'light':
                return A.Compose([
                    A.Resize(Config.INPUT_SIZE, Config.INPUT_SIZE),
                    A.HorizontalFlip(p=0.3),
                    A.RandomRotate90(p=0.3),
                    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
                    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ToTensorV2()
                ])
            elif augmentation_strength == 'heavy':
                return A.Compose([
                    A.Resize(Config.INPUT_SIZE, Config.INPUT_SIZE),
                    A.HorizontalFlip(p=0.7),
                    A.VerticalFlip(p=0.5),
                    A.RandomRotate90(p=0.7),
                    A.ShiftScaleRotate(shift_limit=0.3, scale_limit=0.3, rotate_limit=45, p=0.8),
                    A.RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.4, p=0.8),
                    A.HueSaturationValue(hue_shift_limit=30, sat_shift_limit=40, val_shift_limit=30, p=0.7),
                    A.GaussianBlur(blur_limit=(3, 9), p=0.5),
                    A.GaussNoise(var_limit=(10.0, 80.0), p=0.4),
                    A.CoarseDropout(max_holes=12, max_height=25, max_width=25, p=0.5),
                    A.ElasticTransform(p=0.3),
                    A.GridDistortion(p=0.3),
                    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ToTensorV2()
                ])
            else:  # medium
                return A.Compose([
                    A.Resize(Config.INPUT_SIZE, Config.INPUT_SIZE),
                    A.HorizontalFlip(p=0.5),
                    A.VerticalFlip(p=0.3),
                    A.RandomRotate90(p=0.5),
                    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.7),
                    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
                    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.6),
                    A.GaussianBlur(blur_limit=(3, 7), p=0.4),
                    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
                    A.CoarseDropout(max_holes=8, max_height=20, max_width=20, p=0.4),
                    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ToTensorV2()
                ])
        else:
            return A.Compose([
                A.Resize(Config.INPUT_SIZE, Config.INPUT_SIZE),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])

    @staticmethod
    def load_and_balance_data():
        """Load data and apply SMOTE"""
        print("Loading and preprocessing data...")

        X = np.load(Config.DATA_FILE)
        Y = np.load(Config.LABELS_FILE)

        print(f"Original data shape: {X.shape}")
        print(f"Original class distribution: {np.bincount(Y)}")

        print("Applying SMOTE for class balancing...")
        X_flat = X.reshape(X.shape[0], -1)
        smote = SMOTE(random_state=Config.SEED, k_neighbors=min(5, np.bincount(Y).min()-1))
        X_balanced_flat, Y_balanced = smote.fit_resample(X_flat, Y)
        X_balanced = X_balanced_flat.reshape(-1, *X.shape[1:])

        print(f"Balanced data shape: {X_balanced.shape}")
        print(f"Balanced class distribution: {np.bincount(Y_balanced)}")

        return X_balanced, Y_balanced

    @staticmethod
    def create_data_loaders(X, Y, test_size=0.2, batch_size=None, augmentation_strength='medium'):
        if batch_size is None:
            if torch.cuda.is_available():
                gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
                if gpu_memory_gb >= 24:
                    batch_size = 128
                elif gpu_memory_gb >= 12:
                    batch_size = 96
                elif gpu_memory_gb >= 8:
                    batch_size = 64
                else:
                    batch_size = 48
            else:
                batch_size = Config.BATCH_SIZE

        X_temp, X_test, y_temp, y_test = train_test_split(
            X, Y, test_size=test_size, random_state=Config.SEED, stratify=Y
        )
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=0.25, random_state=Config.SEED, stratify=y_temp
        )

        print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")
        print(f"Using optimized batch size: {batch_size}")

        train_dataset = FishDataset(X_train, y_train,
                                  DataManager.get_transforms(True, augmentation_strength))
        val_dataset = FishDataset(X_val, y_val, DataManager.get_transforms(False))
        test_dataset = FishDataset(X_test, y_test, DataManager.get_transforms(False))

        class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        sample_weights = [class_weights[y] for y in y_train]
        sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=sampler,
            num_workers=Config.DATALOADER_NUM_WORKERS,
            pin_memory=Config.PIN_MEMORY,
            prefetch_factor=2,
            persistent_workers=True
        )
        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=Config.DATALOADER_NUM_WORKERS,
            pin_memory=Config.PIN_MEMORY,
            prefetch_factor=2,
            persistent_workers=True
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=Config.DATALOADER_NUM_WORKERS,
            pin_memory=Config.PIN_MEMORY,
            prefetch_factor=2,
            persistent_workers=True
        )

        return train_loader, val_loader, test_loader, (X_val, y_val), (X_test, y_test)

# ---
# 5. MODEL FACTORY
# =============================================================================
# Purpose: Create different neural network models with customizable architectures.

class ModelFactory:
    @staticmethod
    def create_model(model_name, num_classes=Config.NUM_CLASSES, dropout_rate=0.5,
                    hidden_dim_multiplier=0.5):
        """Create model with configurable architecture"""

        if model_name == 'resnet50':
            model = models.resnet50(weights='IMAGENET1K_V2')
            num_features = model.fc.in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.fc = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'efficientnet_b0':
            model = models.efficientnet_b0(weights='IMAGENET1K_V1')
            num_features = model.classifier[1].in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'mobilenet_v3_large':
            model = models.mobilenet_v3_large(weights='IMAGENET1K_V2')
            num_features = 960
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'vgg16':
            model = models.vgg16(weights='IMAGENET1K_V1')
            hidden_dim = int(4096 * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                nn.Linear(512 * 7 * 7, 4096),
                nn.ReLU(True),
                nn.Dropout(),
                nn.Linear(4096, hidden_dim),
                nn.ReLU(True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'densenet121':
            model = models.densenet121(weights='IMAGENET1K_V1')
            num_features = model.classifier.in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        else:
            raise ValueError(f"Unsupported model: {model_name}")

        return model

# ---
# 6. LEARNABLE WEIGHTED ENSEMBLE MODEL
# =============================================================================
# Purpose: Define a neural network for learning optimal ensemble weights.

class LearnableWeightedEnsemble(nn.Module):
    """Neural network for learning optimal ensemble weights"""
    def __init__(self, num_models, num_classes, hidden_dim=64):
        super(LearnableWeightedEnsemble, self).__init__()
        self.num_models = num_models
        self.num_classes = num_classes

        self.weight_network = nn.Sequential(
            nn.Linear(num_models * num_classes, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, num_models),
            nn.Softmax(dim=1)
        )

        self.prediction_head = nn.Sequential(
            nn.Linear(num_classes, num_classes * 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(num_classes * 2, num_classes)
        )

    def forward(self, model_predictions):
        batch_size = model_predictions.shape[0]

        flattened_preds = model_predictions.view(batch_size, -1)

        weights = self.weight_network(flattened_preds)

        weighted_avg = torch.sum(model_predictions * weights.unsqueeze(-1), dim=1)

        final_predictions = self.prediction_head(weighted_avg)

        return final_predictions, weights

# ---
# 7. HYPERPARAMETER OPTIMIZATION
# =============================================================================
# Purpose: Tune model hyperparameters using Optuna.

class ExpandedHyperparameterOptimizer:
    def __init__(self, model_name, train_loader, val_loader):
        self.model_name = model_name
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.total_batches_per_epoch = len(train_loader)
        self.batch_times = []

    def _update_progress(self, batch_idx, batch_loss, batch_acc, trial_params, is_training=True):
        """Display progress during hyperparameter tuning"""
        batch_time = time.time()
        self.batch_times.append(batch_time)

        if len(self.batch_times) > 1:
            avg_batch_time = np.mean(np.diff(self.batch_times[-10:]))
        else:
            avg_batch_time = 1.0

        remaining_batches = self.total_batches_per_epoch - (batch_idx + 1)
        eta_epoch = remaining_batches * avg_batch_time

        progress_pct = (batch_idx + 1) / self.total_batches_per_epoch * 100
        bar_length = 30
        filled_length = int(bar_length * (batch_idx + 1) // self.total_batches_per_epoch)
        bar = '█' * filled_length + '-' * (bar_length - filled_length)

        eta_str = str(timedelta(seconds=int(eta_epoch)))

        mode = "TRAIN" if is_training else "VAL  "
        print(f"\r{mode} |{bar}| {progress_pct:5.1f}% | "
              f"Batch: {batch_idx+1:4d}/{self.total_batches_per_epoch} | "
              f"Loss: {batch_loss:.4f} | Acc: {batch_acc:.4f} | "
              f"ETA: {eta_str} | Params: {self._format_params(trial_params)}", end='', flush=True)

    def _format_params(self, params):
        """Format hyperparameters for display"""
        formatted = []
        for key, value in params.items():
            if key in ['lr', 'weight_decay']:
                formatted.append(f"{key}: {value:.4f}")
            elif key in ['dropout', 'hidden_dim_multiplier', 'label_smoothing']:
                formatted.append(f"{key}: {value:.1f}")
            else:
                formatted.append(f"{key}: {value}")
        return ", ".join(formatted)

    def objective(self, trial):
        """Optuna objective function with hyperparameters"""
        lr = trial.suggest_float('lr', 1e-5, 5e-3, log=True)
        weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True)
        dropout = trial.suggest_float('dropout', 0.2, 0.8)
        hidden_dim_multiplier = trial.suggest_float('hidden_dim_multiplier', 0.25, 1.0)
        augmentation_strength = trial.suggest_categorical('augmentation_strength',
                                                        ['light', 'medium', 'heavy'])
        batch_size = trial.suggest_categorical('batch_size', [16, 24, 32, 48, 64])
        optimizer_type = trial.suggest_categorical('optimizer_type', ['adam', 'adamw', 'sgd'])
        scheduler_type = trial.suggest_categorical('scheduler_type',
                                                 ['cosine', 'step', 'exponential'])
        label_smoothing = trial.suggest_float('label_smoothing', 0.0, 0.2)

        trial_params = {
            'lr': lr, 'weight_decay': weight_decay, 'dropout': dropout,
            'hidden_dim_multiplier': hidden_dim_multiplier, 'augmentation_strength': augmentation_strength,
            'batch_size': batch_size, 'optimizer_type': optimizer_type, 'scheduler_type': scheduler_type,
            'label_smoothing': label_smoothing
        }

        try:
            print(f"\nTrial {trial.number} parameters:")
            for key, value in trial_params.items():
                if key in ['lr', 'weight_decay']:
                    print(f"  {key}: {value:.4f}")
                elif key in ['dropout', 'hidden_dim_multiplier', 'label_smoothing']:
                    print(f"  {key}: {value:.1f}")
                else:
                    print(f"  {key}: {value}")

            temp_train_loader = DataLoader(
                self.train_loader.dataset,
                batch_size=batch_size,
                sampler=self.train_loader.sampler
            )
            temp_val_loader = DataLoader(
                self.val_loader.dataset,
                batch_size=batch_size,
                shuffle=False
            )

            model = ModelFactory.create_model(
                self.model_name,
                dropout_rate=dropout,
                hidden_dim_multiplier=hidden_dim_multiplier
            )
            model = model.to(Config.DEVICE)

            if optimizer_type == 'adam':
                optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
            elif optimizer_type == 'adamw':
                optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
            else:
                optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay,
                                    momentum=0.9, nesterov=True)

            criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

            if scheduler_type == 'cosine':
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.OPTUNA_EPOCHS)
            elif scheduler_type == 'step':
                scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.7)
            else:
                scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

            best_val_acc = 0
            patience_counter = 0

            for epoch in range(Config.OPTUNA_EPOCHS):
                self.batch_times = []
                model.train()
                train_correct = 0
                train_total = 0
                train_loss = 0

                for batch_idx, (images, labels) in enumerate(temp_train_loader):
                    if batch_idx > 15:
                        break
                    images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()

                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    optimizer.step()

                    batch_loss = loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    train_total += labels.size(0)
                    train_correct += (predicted == labels).sum().item()
                    train_loss += batch_loss

                    batch_acc = train_correct / train_total
                    self._update_progress(batch_idx, batch_loss, batch_acc, trial_params, is_training=True)

                model.eval()
                val_correct = 0
                val_total = 0
                val_loss = 0
                val_predictions = []
                val_labels = []

                with torch.no_grad():
                    for batch_idx, (images, labels) in enumerate(temp_val_loader):
                        if batch_idx > 8:
                            break
                        images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
                        outputs = model(images)
                        loss = criterion(outputs, labels)
                        batch_loss = loss.item()
                        val_loss += batch_loss
                        _, predicted = torch.max(outputs.data, 1)
                        val_total += labels.size(0)
                        val_correct += (predicted == labels).sum().item()
                        val_predictions.extend(predicted.cpu().numpy())
                        val_labels.extend(labels.cpu().numpy())

                        batch_acc = val_correct / val_total
                        self._update_progress(batch_idx, batch_loss, batch_acc, trial_params, is_training=False)

                train_acc = train_correct / train_total
                val_acc = val_correct / val_total
                val_f1 = f1_score(val_labels, val_predictions, average='macro')
                scheduler.step()

                print(f"\nEpoch {epoch+1}/{Config.OPTUNA_EPOCHS} Summary: "
                      f"Train Loss: {train_loss/len(temp_train_loader):.6f}, "
                      f"Train Acc: {train_acc:.4f}, Val Loss: {val_loss/len(temp_val_loader):.6f}, "
                      f"Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    patience_counter = 0
                else:
                    patience_counter += 1

                if patience_counter >= 4:
                    print(f"\nEarly stopping at epoch {epoch+1}")
                    break

                trial.report(val_acc, epoch)
                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()

            return best_val_acc

        except Exception as e:
            print(f"\nTrial failed: {e}")
            return 0.0

    def optimize(self):
        print(f"Optimizing hyperparameters for {self.model_name}...")

        if not OPTUNA_AVAILABLE:
            return {
                'lr': Config.LEARNING_RATE,
                'dropout': 0.5,
                'weight_decay': Config.WEIGHT_DECAY,
                'hidden_dim_multiplier': 0.5,
                'augmentation_strength': 'medium',
                'batch_size': Config.BATCH_SIZE,
                'optimizer_type': 'adamw',
                'scheduler_type': 'cosine',
                'label_smoothing': 0.1
            }

        study = optuna.create_study(direction='maximize')
        study.optimize(self.objective, n_trials=Config.OPTUNA_TRIALS, timeout=1200)

        best_params = study.best_trial.params
        print(f"\nBest params for {self.model_name}:")
        for key, value in best_params.items():
            if key in ['lr', 'weight_decay']:
                print(f"  {key}: {value:.4f}")
            elif key in ['dropout', 'hidden_dim_multiplier', 'label_smoothing']:
                print(f"  {key}: {value:.1f}")
            else:
                print(f"  {key}: {value}")
        print(f"Best validation accuracy: {study.best_trial.value:.4f}")

        return best_params

# ---
# 8. MODEL TRAINING
# =============================================================================
# Purpose: Train individual models with optimized hyperparameters.

class EnhancedModelTrainer:
    def __init__(self, model, model_name, hyperparameters):
        self.model = model.to(Config.DEVICE)
        self.model_name = model_name
        self.hyperparameters = hyperparameters
        self.best_val_acc = 0.0
        self.best_val_f1 = 0.0
        self.patience_counter = 0
        self.history = {
            'train_loss': [], 'train_acc': [],
            'val_loss': [], 'val_acc': [], 'val_f1': [],
            'learning_rates': []
        }
        # Create model directory
        os.makedirs(f"{Config.OUTPUT_DIR}/models", exist_ok=True)
        self._setup_training_components()

    def _setup_training_components(self):
        lr = self.hyperparameters.get('lr', Config.LEARNING_RATE)
        weight_decay = self.hyperparameters.get('weight_decay', Config.WEIGHT_DECAY)
        optimizer_type = self.hyperparameters.get('optimizer_type', 'adamw')

        if optimizer_type == 'adam':
            self.optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=weight_decay)
        elif optimizer_type == 'adamw':
            self.optimizer = optim.AdamW(self.model.parameters(), lr=lr, weight_decay=weight_decay)
        else:
            self.optimizer = optim.SGD(self.model.parameters(), lr=lr, weight_decay=weight_decay,
                                     momentum=0.9, nesterov=True)

        label_smoothing = self.hyperparameters.get('label_smoothing', 0.1)
        self.criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

        scheduler_type = self.hyperparameters.get('scheduler_type', 'cosine')
        if scheduler_type == 'cosine':
            self.scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
                self.optimizer, T_0=10, T_mult=2
            )
        elif scheduler_type == 'step':
            self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.5)
        else:
            self.scheduler = optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.95)

    def train_epoch(self, train_loader, progress_tracker):
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0
        batch_losses = []
        batch_accuracies = []

        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()

            batch_loss = loss.item()
            _, predicted = torch.max(outputs.data, 1)
            batch_total = labels.size(0)
            batch_correct = (predicted == labels).sum().item()
            batch_acc = batch_correct / batch_total

            total_loss += batch_loss
            total += batch_total
            correct += batch_correct
            batch_losses.append(batch_loss)
            batch_accuracies.append(batch_acc)

            progress_tracker.update_batch(batch_idx, batch_loss, batch_acc, is_training=True)

        return total_loss / len(train_loader), correct / total

    def validate_epoch(self, val_loader, progress_tracker):
        self.model.eval()
        total_loss = 0
        all_predictions = []
        all_labels = []

        with torch.no_grad():
            for batch_idx, (images, labels) in enumerate(val_loader):
                images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                batch_loss = loss.item()
                total_loss += batch_loss
                _, predicted = torch.max(outputs.data, 1)
                batch_acc = (predicted == labels).float().mean().item()

                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                progress_tracker.update_batch(batch_idx, batch_loss, batch_acc, is_training=False)

        if not all_labels:
            print(f"Warning: No validation data processed for {self.model_name}")
            return float('inf'), 0.0, 0.0

        accuracy = accuracy_score(all_labels, all_predictions)
        f1 = f1_score(all_labels, all_predictions, average='macro')
        return total_loss / len(val_loader), accuracy, f1

    def train(self, train_loader, val_loader):
        print(f"Training {self.model_name} with hyperparameters:")
        for key, value in self.hyperparameters.items():
            if key in ['lr', 'weight_decay', 'dropout', 'hidden_dim_multiplier', 'label_smoothing']:
                print(f"  {key}: {value:.4f}")
            else:
                print(f"  {key}: {value}")

        progress_tracker = TrainingProgressTracker(
            self.model_name, Config.EPOCHS, len(train_loader)
        )

        for epoch in range(Config.EPOCHS):
            progress_tracker.start_epoch(epoch)
            train_loss, train_acc = self.train_epoch(train_loader, progress_tracker)
            val_loss, val_acc, val_f1 = self.validate_epoch(val_loader, progress_tracker)
            self.scheduler.step()
            current_lr = self.optimizer.param_groups[0]['lr']

            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            self.history['val_f1'].append(val_f1)
            self.history['learning_rates'].append(current_lr)

            is_best = False
            if val_f1 > self.best_val_f1:
                self.best_val_f1 = val_f1
                self.best_val_acc = val_acc
                self.patience_counter = 0
                is_best = True
                torch.save(self.model.state_dict(),
                          f"{Config.OUTPUT_DIR}/models/{self.model_name}_best.pt")
            else:
                self.patience_counter += 1

            progress_tracker.finish_epoch(
                train_loss, train_acc, val_loss, val_acc, val_f1,
                is_best=is_best, lr=current_lr
            )

            if self.patience_counter >= Config.PATIENCE:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

        print(f"History for {self.model_name}:")
        print(f"  train_loss: {len(self.history['train_loss'])} entries")
        print(f"  train_acc: {len(self.history['train_acc'])} entries")
        print(f"  val_loss: {len(self.history['val_loss'])} entries")
        print(f"  val_acc: {len(self.history['val_acc'])} entries")
        print(f"  val_f1: {len(self.history['val_f1'])} entries")

        self.model.load_state_dict(
            torch.load(f"{Config.OUTPUT_DIR}/models/{self.model_name}_best.pt")
        )

        print(f"\n✓ {self.model_name} training completed!")
        print(f"  Best Validation F1: {self.best_val_f1:.4f}")
        print(f"  Best Validation Accuracy: {self.best_val_acc:.4f}")
        print("="*60)

        return self.history

# ---
# 9. ENSEMBLE METHODS
# =============================================================================
# Purpose: Implement ensemble methods (simple, weighted, confidence-based, learnable).

class EnsembleManager:
    def __init__(self, models_dict, val_data):
        self.models = models_dict
        self.X_val, self.y_val = val_data
        self.model_predictions = self._get_predictions()
        self.histories = {}

    def _get_predictions(self):
        print("Getting model predictions for ensemble...")
        predictions = {}

        val_dataset = FishDataset(self.X_val, self.y_val, DataManager.get_transforms(False))
        val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)

        for name, model in self.models.items():
            model.eval()
            all_preds = []
            all_probs = []
            all_losses = []
            all_labels = []
            total = 0
            correct = 0
            criterion = nn.CrossEntropyLoss()

            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
                    outputs = model(images)
                    probabilities = torch.softmax(outputs, dim=1)
                    loss = criterion(outputs, labels).item()

                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    all_preds.extend(predicted.cpu().numpy())
                    all_probs.extend(probabilities.cpu().numpy())
                    all_losses.append(loss)
                    all_labels.extend(labels.cpu().numpy())

            accuracy = correct / total
            f1 = f1_score(self.y_val, all_preds, average='macro')
            avg_loss = np.mean(all_losses)

            predictions[name] = {
                'predictions': np.array(all_preds),
                'probabilities': np.array(all_probs),
                'loss': avg_loss,
                'accuracy': accuracy,
                'f1': f1,
                'true_labels': np.array(all_labels)
            }

            print(f"  {name}: F1 = {f1:.4f}, Acc = {accuracy:.4f}, Loss = {avg_loss:.4f}")

        return predictions

    def simple_average_ensemble(self, model_combo):
        selected_probs = [self.model_predictions[name]['probabilities'] for name in model_combo]
        avg_probs = np.mean(selected_probs, axis=0)
        predictions = np.argmax(avg_probs, axis=1)

        accuracy = accuracy_score(self.y_val, predictions)
        f1 = f1_score(self.y_val, predictions, average='macro')
        loss = np.mean([self.model_predictions[name]['loss'] for name in model_combo])

        return {
            'accuracy': accuracy,
            'f1': f1,
            'loss': loss,
            'predictions': predictions,
            'models': model_combo,
            'probabilities': avg_probs,
            'true_labels': self.y_val
        }

    def weighted_average_ensemble(self, model_combo):
        weights = []
        selected_probs = []

        for name in model_combo:
            f1 = self.model_predictions[name]['f1']
            weights.append(f1)
            selected_probs.append(self.model_predictions[name]['probabilities'])

        weights = np.array(weights) / np.sum(weights)
        weighted_probs = np.average(selected_probs, axis=0, weights=weights)
        predictions = np.argmax(weighted_probs, axis=1)

        accuracy = accuracy_score(self.y_val, predictions)
        f1 = f1_score(self.y_val, predictions, average='macro')
        loss = np.average([self.model_predictions[name]['loss'] for name in model_combo], weights=weights)

        return {
            'accuracy': accuracy,
            'f1': f1,
            'loss': loss,
            'predictions': predictions,
            'weights': weights,
            'models': model_combo,
            'probabilities': weighted_probs,
            'true_labels': self.y_val
        }

    def confidence_based_ensemble(self, model_combo):
        final_predictions = []
        all_probs = []

        for i in range(len(self.y_val)):
            confidences = []
            probs = []

            for name in model_combo:
                prob = self.model_predictions[name]['probabilities'][i]
                confidence = np.max(prob)
                confidences.append(confidence)
                probs.append(prob)

            confidences = np.array(confidences)
            weights = confidences / np.sum(confidences) if np.sum(confidences) > 0 else np.ones(len(confidences)) / len(confidences)

            final_prob = np.average(probs, axis=0, weights=weights)
            final_predictions.append(np.argmax(final_prob))
            all_probs.append(final_prob)

        predictions = np.array(final_predictions)
        accuracy = accuracy_score(self.y_val, predictions)
        f1 = f1_score(self.y_val, predictions, average='macro')
        loss = np.mean([self.model_predictions[name]['loss'] for name in model_combo])

        return {
            'accuracy': accuracy,
            'f1': f1,
            'loss': loss,
            'predictions': predictions,
            'models': model_combo,
            'probabilities': np.array(all_probs),
            'true_labels': self.y_val
        }

    def learnable_weighted_ensemble(self, model_combo, epochs=30):
        print(f"Training learnable weighted ensemble with {len(model_combo)} models...")

        selected_probs = []
        for name in model_combo:
            selected_probs.append(self.model_predictions[name]['probabilities'])

        ensemble_input = np.stack(selected_probs, axis=1)

        X_ensemble = torch.FloatTensor(ensemble_input).to(Config.DEVICE)
        y_ensemble = torch.LongTensor(self.y_val).to(Config.DEVICE)

        ensemble_model = LearnableWeightedEnsemble(
            num_models=len(model_combo),
            num_classes=Config.NUM_CLASSES
        ).to(Config.DEVICE)

        optimizer = optim.AdamW(ensemble_model.parameters(), lr=1e-3, weight_decay=1e-4)
        criterion = nn.CrossEntropyLoss()
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

        history = {'train_loss': [], 'train_acc': [], 'val_f1': []}
        best_loss = float('inf')

        for epoch in range(epochs):
            ensemble_model.train()
            optimizer.zero_grad()
            predictions, weights = ensemble_model(X_ensemble)
            loss = criterion(predictions, y_ensemble)
            loss.backward()
            optimizer.step()
            scheduler.step()

            accuracy = accuracy_score(y_ensemble.cpu().numpy(), predictions.argmax(dim=1).cpu().numpy())
            f1 = f1_score(y_ensemble.cpu().numpy(), predictions.argmax(dim=1).cpu().numpy(), average='macro')

            history['train_loss'].append(loss.item())
            history['train_acc'].append(accuracy)
            history['val_f1'].append(f1)

            print(f"Ensemble Epoch {epoch+1}/{epochs}: Loss = {loss.item():.4f}, "
                  f"Acc = {accuracy:.4f}, F1 = {f1:.4f}")

            if loss.item() < best_loss:
                best_loss = loss.item()
                torch.save(ensemble_model.state_dict(), f"{Config.OUTPUT_DIR}/models/learnable_ensemble_{'+'.join(model_combo)}.pt")

        ensemble_model.load_state_dict(torch.load(f"{Config.OUTPUT_DIR}/models/learnable_ensemble_{'+'.join(model_combo)}.pt"))
        ensemble_model.eval()
        with torch.no_grad():
            final_predictions, learned_weights = ensemble_model(X_ensemble)
            predictions = final_predictions.argmax(dim=1).cpu().numpy()
            probabilities = torch.softmax(final_predictions, dim=1).cpu().numpy()
            avg_weights = learned_weights.mean(dim=0).cpu().numpy()

        accuracy = accuracy_score(self.y_val, predictions)
        f1 = f1_score(self.y_val, predictions, average='macro')
        loss = np.mean([self.model_predictions[name]['loss'] for name in model_combo])

        self.histories[f"learnable_weighted_{'+'.join(model_combo)}"] = history

        return {
            'accuracy': accuracy,
            'f1': f1,
            'loss': loss,
            'predictions': predictions,
            'models': model_combo,
            'learned_weights': avg_weights,
            'probabilities': probabilities,
            'true_labels': self.y_val
        }

    def test_ensemble_combinations(self):
        print("Testing ensemble combinations...")

        model_names = list(self.models.keys())
        all_results = {}
        best_result = None
        best_score = 0

        for size in range(2, min(len(model_names) + 1, 5)):
            print(f"Testing {size}-model combinations...")

            for combo in list(combinations(model_names, size))[:5]:
                combo_name = f"combo_{size}_{'+'.join(combo)}"

                for method_name in Config.ENSEMBLE_METHODS:
                    full_name = f"{combo_name}_{method_name}"

                    try:
                        if method_name == 'simple_average':
                            result = self.simple_average_ensemble(combo)
                        elif method_name == 'weighted_average':
                            result = self.weighted_average_ensemble(combo)
                        elif method_name == 'confidence_based':
                            result = self.confidence_based_ensemble(combo)
                        elif method_name == 'learnable_weighted':
                            result = self.learnable_weighted_ensemble(combo)

                        # Verify result contains required keys
                        required_keys = ['accuracy', 'f1', 'loss', 'predictions', 'models', 'probabilities', 'true_labels']
                        if not all(key in result for key in required_keys):
                            missing = [key for key in required_keys if key not in result]
                            print(f"  {full_name}: Missing keys {missing}")
                            continue

                        all_results[full_name] = result
                        print(f"  {full_name}: F1 = {result['f1']:.4f}, Acc = {result['accuracy']:.4f}, "
                              f"Loss = {result['loss']:.4f}, True Labels Shape = {result['true_labels'].shape}")

                        if result['f1'] > best_score:
                            best_score = result['f1']
                            best_result = (full_name, result)

                    except Exception as e:
                        print(f"  {full_name}: FAILED - {str(e)}")

        if best_result:
            print(f"\n✓ Best ensemble: {best_result[0]} (F1: {best_result[1]['f1']:.4f})")
        else:
            print("\nNo valid ensemble results generated.")

        return all_results, best_result

# ---
# 10. MODEL EVALUATION
# =============================================================================
# Purpose: Evaluate models on test data.

class ModelEvaluator:
    def evaluate_model(self, model, test_loader, model_name):
        print(f"Evaluating {model_name}...")
        model.eval()
        all_preds = []
        all_labels = []
        all_probs = []
        total_loss = 0
        criterion = nn.CrossEntropyLoss()

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
                outputs = model(images)
                loss = criterion(outputs, labels).item()
                probabilities = torch.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs.data, 1)

                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probabilities.cpu().numpy())
                total_loss += loss

        if not all_labels:
            print(f"Warning: No test data processed for {model_name}")
            return {
                'model_name': model_name,
                'accuracy': 0.0,
                'f1_macro': 0.0,
                'f1_weighted': 0.0,
                'precision_macro': 0.0,
                'recall_macro': 0.0,
                'f1_per_class': np.zeros(Config.NUM_CLASSES),
                'precision_per_class': np.zeros(Config.NUM_CLASSES),
                'recall_per_class': np.zeros(Config.NUM_CLASSES),
                'predictions': np.array([]),
                'true_labels': np.array([]),
                'probabilities': np.array([]),
                'loss': float('inf')
            }

        accuracy = accuracy_score(all_labels, all_preds)
        f1_macro = f1_score(all_labels, all_preds, average='macro')
        f1_weighted = f1_score(all_labels, all_preds, average='weighted')
        precision_macro = precision_score(all_labels, all_preds, average='macro')
        recall_macro = recall_score(all_labels, all_preds, average='macro')
        f1_per_class = f1_score(all_labels, all_preds, average=None)
        precision_per_class = precision_score(all_labels, all_preds, average=None)
        recall_per_class = recall_score(all_labels, all_preds, average=None)
        avg_loss = total_loss / len(test_loader)

        print(f"{model_name} Evaluation: Acc = {accuracy:.4f}, F1 (Macro) = {f1_macro:.4f}, Loss = {avg_loss:.4f}")

        return {
            'model_name': model_name,
            'accuracy': accuracy,
            'f1_macro': f1_macro,
            'f1_weighted': f1_weighted,
            'precision_macro': precision_macro,
            'recall_macro': recall_macro,
            'f1_per_class': f1_per_class,
            'precision_per_class': precision_per_class,
            'recall_per_class': recall_per_class,
            'predictions': np.array(all_preds),
            'true_labels': np.array(all_labels),
            'probabilities': np.array(all_probs),
            'loss': avg_loss
        }

# ---
# 11. ENHANCED VISUALIZATIONS
# =============================================================================
# Purpose: Generate visualizations for training history, ROC curves, confusion matrices, model comparisons, and XAI visualizations.

class EnhancedVisualizations:
    def __init__(self):
        self.viz_dir = f"{Config.OUTPUT_DIR}/visualizations"
        Path(self.viz_dir).mkdir(parents=True, exist_ok=True)

    def plot_training_history(self, histories, model_type='single'):
        for model_name, history in histories.items():
            print(f"Plotting history for {model_name}:")
            print(f"  train_loss: {len(history['train_loss'])} entries")
            print(f"  train_acc: {len(history['train_acc'])} entries")
            print(f"  val_loss: {len(history.get('val_loss', []))} entries")
            print(f"  val_acc: {len(history.get('val_acc', []))} entries")
            print(f"  val_f1: {len(history.get('val_f1', []))} entries")

            if not history['train_loss']:
                print(f"Skipping {model_name}: No training data available")
                continue

            epochs = range(1, len(history['train_loss']) + 1)

            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

            # Plot training loss
            ax1.plot(epochs, history['train_loss'], label='Train Loss', color='blue')
            if history.get('val_loss', []):
                if len(history['val_loss']) == len(history['train_loss']):
                    ax1.plot(epochs, history['val_loss'], label='Val Loss', color='orange')
                else:
                    print(f"Warning: val_loss length ({len(history['val_loss'])}) does not match train_loss length ({len(history['train_loss'])}) for {model_name}")
            ax1.set_title(f'{model_name} - Loss vs Epoch')
            ax1.set_xlabel('Epoch')
            ax1.set_ylabel('Loss')
            ax1.legend()
            ax1.grid(True)

            # Plot training accuracy
            ax2.plot(epochs, history['train_acc'], label='Train Acc', color='green')
            if history.get('val_acc', []):
                if len(history['val_acc']) == len(history['train_acc']):
                    ax2.plot(epochs, history['val_acc'], label='Val Acc', color='red')
                else:
                    print(f"Warning: val_acc length ({len(history['val_acc'])}) does not match train_acc length ({len(history['train_acc'])}) for {model_name}")
            ax2.set_title(f'{model_name} - Accuracy vs Epoch')
            ax2.set_xlabel('Epoch')
            ax2.set_ylabel('Accuracy')
            ax2.legend()
            ax2.grid(True)

            plt.suptitle(f'{model_type.capitalize()} Model: {model_name}', fontsize=16)
            plt.tight_layout()
            save_path = f"{self.viz_dir}/{model_name}_training_history.png"
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.show()
            print(f"Training history plot saved: {save_path}")

    def plot_roc_curves(self, results, model_name):
        fig, ax = plt.subplots(figsize=(10, 6))
        if 'true_labels' not in results or 'probabilities' not in results:
            print(f"Error: Missing 'true_labels' or 'probabilities' in results for {model_name}")
            return
        for i in range(Config.NUM_CLASSES):
            fpr, tpr, _ = roc_curve(np.array(results['true_labels']) == i, results['probabilities'][:, i])
            roc_auc = auc(fpr, tpr)
            ax.plot(fpr, tpr, label=f'Class {Config.CLASS_LABELS[i]} (AUC = {roc_auc:.2f})')
        ax.plot([0, 1], [0, 1], 'k--')
        ax.set_title(f'{model_name} - ROC Curve')
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.legend()
        ax.grid(True)
        plt.show()

    def plot_confusion_matrix(self, results, model_name):
        if 'true_labels' not in results or 'predictions' not in results:
            print(f"Error: Missing 'true_labels' or 'predictions' in results for {model_name}")
            return
        cm = confusion_matrix(results['true_labels'], results['predictions'])
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        fig, ax = plt.subplots(figsize=(10, 8))
        sns.heatmap(cm_normalized, annot=cm, fmt='d', cmap='Blues',
                    xticklabels=Config.CLASS_LABELS, yticklabels=Config.CLASS_LABELS,
                    cbar_kws={'label': 'Normalized Count'})
        ax.set_title(f'Confusion Matrix: {model_name}')
        ax.set_xlabel('Predicted Label')
        ax.set_ylabel('True Label')
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()

        save_path = f"{self.viz_dir}/{model_name}_confusion_matrix.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        print(f"Confusion matrix saved: {save_path}")

    def plot_model_comparison(self, single_results, ensemble_results):
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

        model_names = list(single_results.keys())
        accuracies = [single_results[name]['accuracy'] for name in model_names]
        f1_scores = [single_results[name]['f1_macro'] for name in model_names]
        f1_weighted = [single_results[name]['f1_weighted'] for name in model_names]
        losses = [single_results[name]['loss'] for name in model_names]

        x = np.arange(len(model_names))
        width = 0.2

        ax1.bar(x - width*1.5, accuracies, width, label='Accuracy', alpha=0.8, color='skyblue')
        ax1.bar(x - width/2, f1_scores, width, label='F1 (Macro)', alpha=0.8, color='lightcoral')
        ax1.bar(x + width/2, f1_weighted, width, label='F1 (Weighted)', alpha=0.8, color='lightgreen')
        ax1.bar(x + width*1.5, losses, width, label='Loss', alpha=0.8, color='salmon')

        ax1.set_xlabel('Models', fontweight='bold')
        ax1.set_ylabel('Score', fontweight='bold')
        ax1.set_title('Individual Model Performance', fontweight='bold', fontsize=14)
        ax1.set_xticks(x)
        ax1.set_xticklabels(model_names, rotation=45, ha='right')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.set_ylim(0, max(max(accuracies), max(f1_scores), max(f1_weighted), max(losses)) * 1.1)

        if ensemble_results:
            ensemble_items = list(ensemble_results.items())
            ensemble_items.sort(key=lambda x: x[1]['f1'], reverse=True)
            top_ensembles = ensemble_items[:10]

            ensemble_names = [name.split('_')[-1] for name, _ in top_ensembles]
            ensemble_f1s = [result['f1'] for _, result in top_ensembles]
            ensemble_accs = [result['accuracy'] for _, result in top_ensembles]
            ensemble_losses = [result['loss'] for _, result in top_ensembles]

            x = np.arange(len(ensemble_names))
            ax2.bar(x - width, ensemble_accs, width, label='Accuracy', alpha=0.8, color='skyblue')
            ax2.bar(x, ensemble_f1s, width, label='F1 (Macro)', alpha=0.8, color='lightcoral')
            ax2.bar(x + width, ensemble_losses, width, label='Loss', alpha=0.8, color='salmon')

            ax2.set_xticks(x)
            ax2.set_xticklabels([f"{name}" for name in ensemble_names], rotation=45, ha='right')
            ax2.set_xlabel('Ensemble Methods', fontweight='bold')
            ax2.set_ylabel('Score', fontweight='bold')
            ax2.set_title('Top Ensemble Methods', fontweight='bold', fontsize=14)
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            ax2.set_ylim(0, max(max(ensemble_accs), max(ensemble_f1s), max(ensemble_losses)) * 1.1)

        if single_results:
            per_class_f1 = []
            for model_name in model_names:
                per_class_f1.append(single_results[model_name]['f1_per_class'])

            per_class_f1 = np.array(per_class_f1)
            im = ax3.imshow(per_class_f1, cmap='RdYlBu_r', aspect='auto', vmin=0, vmax=1)

            ax3.set_xticks(range(len(Config.CLASS_LABELS)))
            ax3.set_xticklabels(Config.CLASS_LABELS, rotation=45, ha='right')
            ax3.set_yticks(range(len(model_names)))
            ax3.set_yticklabels(model_names)
            ax3.set_title('Per-Class F1 Scores', fontweight='bold', fontsize=14)

            cbar = plt.colorbar(im, ax=ax3)
            cbar.set_label('F1 Score', rotation=270, labelpad=15)

            for i in range(len(model_names)):
                for j in range(len(Config.CLASS_LABELS)):
                    ax3.text(j, i, f'{per_class_f1[i, j]:.2f}',
                            ha="center", va="center", color="black", fontsize=8)

        if ensemble_results:
            best_single_f1 = max(f1_scores)
            best_single_name = model_names[f1_scores.index(best_single_f1)]
            best_ensemble_f1 = max([result['f1'] for result in ensemble_results.values()])
            best_ensemble_name = max(ensemble_results.items(), key=lambda x: x[1]['f1'])[0]

            categories = ['Best Single\nModel', 'Best Ensemble']
            values = [best_single_f1, best_ensemble_f1]
            colors = ['lightblue', 'gold']

            bars = ax4.bar(categories, values, color=colors, alpha=0.8)
            ax4.set_ylabel('F1 Score', fontweight='bold')
            ax4.set_title('Single vs Ensemble Comparison', fontweight='bold', fontsize=14)
            ax4.grid(True, alpha=0.3)
            ax4.set_ylim(0, 1)

            for bar, value in zip(bars, values):
                ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f'{value:.4f}', ha='center', va='bottom', fontweight='bold')

            improvement = ((best_ensemble_f1 - best_single_f1) / best_single_f1) * 100
            ax4.text(0.5, 0.5, f'Improvement:\n{improvement:.2f}%',
                    transform=ax4.transAxes, ha='center', va='center',
                    fontsize=12, fontweight='bold',
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7))

            ax4.text(0, best_single_f1/2, best_single_name, ha='center', va='center',
                    rotation=90, fontweight='bold')

        plt.tight_layout()
        save_path = f"{self.viz_dir}/model_comparison.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        print(f"Model comparison saved: {save_path}")
        return save_path

    def plot_gradcam_lrp(self, models_dict, ensemble_results, test_loader, model_type='single'):
        """Generate and save Grad-CAM++ and LRP visualizations for single or ensemble models."""
        xai_visualizer = XAIVisualizer()
        save_dir = f"{self.viz_dir}/{model_type}_xai"
        Path(save_dir).mkdir(parents=True, exist_ok=True)

        # Get a small sample from test_loader
        sample_images, sample_labels = next(iter(test_loader))
        sample_images = sample_images[:5].to(Config.DEVICE)  # Limit to 5 images
        sample_labels = sample_labels[:5].numpy()

        if model_type == 'single':
            for model_name, model in models_dict.items():
                model.eval()
                for idx, (image, true_label) in enumerate(zip(sample_images, sample_labels)):
                    with torch.no_grad():
                        outputs = model(image.unsqueeze(0))
                        probabilities = torch.softmax(outputs, dim=1)
                        predicted_class = outputs.argmax(dim=1).item()
                        confidence = probabilities[0, predicted_class].item()
                        predicted_label = Config.CLASS_LABELS[predicted_class]

                    try:
                        # Grad-CAM++
                        gradcam_img, gradcam_heatmap = xai_visualizer.grad_cam_plus_plus(model, image, predicted_class)
                        # LRP (using integrated gradients)
                        lrp_img, lrp_heatmap = xai_visualizer.lrp(model, image, predicted_class)

                        # Plot and save
                        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
                        image_np = image.permute(1, 2, 0).cpu().numpy() * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                        image_np = np.clip(image_np, 0, 1)
                        image_np = (image_np * 255).astype(np.uint8)

                        ax1.imshow(image_np)
                        ax1.set_title(f'Original Image\nTrue: {Config.CLASS_LABELS[true_label]}\nPredicted: {predicted_label} ({confidence:.2%})')
                        ax1.axis('off')

                        ax2.imshow(gradcam_img)
                        ax2.set_title(f'{model_name} Grad-CAM++')
                        ax2.axis('off')

                        ax3.imshow(lrp_img)
                        ax3.set_title(f'{model_name} Integrated Gradients')
                        ax3.axis('off')

                        plt.suptitle(f'{model_name} XAI Visualizations - Image {idx+1}', fontsize=16)
                        save_path = f"{save_dir}/{model_name}_image_{idx+1}_xai.png"
                        plt.savefig(save_path, dpi=300, bbox_inches='tight')
                        plt.close()
                        print(f"XAI visualization saved: {save_path}")

                    except Exception as e:
                        print(f"Error generating XAI for {model_name}, image {idx+1}: {e}")
                        continue

        elif model_type == 'ensemble' and ensemble_results:
            for ensemble_name, result in list(ensemble_results.items())[:5]:  # Limit to top 5 ensembles
                model_combo = result['models']
                for idx, (image, true_label) in enumerate(zip(sample_images, sample_labels)):
                    try:
                        gradcam_heatmaps = []
                        lrp_heatmaps = []
                        model_predictions = []

                        for model_name in model_combo:
                            model = models_dict[model_name]
                            model.eval()
                            with torch.no_grad():
                                outputs = model(image.unsqueeze(0))
                                probabilities = torch.softmax(outputs, dim=1)
                                model_predictions.append(probabilities.cpu().numpy()[0])

                            predicted_class = np.argmax(probabilities.cpu().numpy()[0])
                            gradcam_img, gradcam_heatmap = xai_visualizer.grad_cam_plus_plus(model, image, predicted_class)
                            lrp_img, lrp_heatmap = xai_visualizer.lrp(model, image, predicted_class)

                            # Extract heatmap values for averaging
                            gradcam_heatmaps.append(cv2.cvtColor(gradcam_heatmap, cv2.COLOR_BGR2RGB))
                            lrp_heatmaps.append(cv2.cvtColor(lrp_heatmap, cv2.COLOR_BGR2RGB))

                        # Average heatmaps for ensemble
                        avg_gradcam = np.mean(gradcam_heatmaps, axis=0).astype(np.uint8)
                        avg_lrp = np.mean(lrp_heatmaps, axis=0).astype(np.uint8)

                        # Ensemble prediction
                        final_probs = np.mean(model_predictions, axis=0)  # Simple average for visualization
                        predicted_class = np.argmax(final_probs)
                        predicted_label = Config.CLASS_LABELS[predicted_class]
                        confidence = final_probs[predicted_class]

                        image_np = image.permute(1, 2, 0).cpu().numpy() * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                        image_np = np.clip(image_np, 0, 1)
                        image_np = (image_np * 255).astype(np.uint8)

                        superimposed_gradcam = cv2.addWeighted(image_np, 0.6, avg_gradcam, 0.4, 0)
                        superimposed_lrp = cv2.addWeighted(image_np, 0.6, avg_lrp, 0.4, 0)

                        # Plot and save
                        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
                        ax1.imshow(image_np)
                        ax1.set_title(f'Original Image\nTrue: {Config.CLASS_LABELS[true_label]}\nPredicted: {predicted_label} ({confidence:.2%})')
                        ax1.axis('off')

                        ax2.imshow(superimposed_gradcam)
                        ax2.set_title(f'{ensemble_name} Grad-CAM++')
                        ax2.axis('off')

                        ax3.imshow(superimposed_lrp)
                        ax3.set_title(f'{ensemble_name} Integrated Gradients')
                        ax3.axis('off')

                        plt.suptitle(f'Ensemble ({ensemble_name}) XAI Visualizations - Image {idx+1}', fontsize=16)
                        save_path = f"{save_dir}/{ensemble_name}_image_{idx+1}_xai.png"
                        plt.savefig(save_path, dpi=300, bbox_inches='tight')
                        plt.close()
                        print(f"Ensemble XAI visualization saved: {save_path}")

                    except Exception as e:
                        print(f"Error generating ensemble XAI for {ensemble_name}, image {idx+1}: {e}")
                        continue

    def plot_comparative_xai(self, models_dict, ensemble_results, test_loader, max_images=3):
        """Generate comparative XAI visualizations across all models for the same images."""
        xai_visualizer = XAIVisualizer()
        save_dir = f"{self.viz_dir}/comparative_xai"
        Path(save_dir).mkdir(parents=True, exist_ok=True)

        # Get a small sample from test_loader
        sample_images, sample_labels = next(iter(test_loader))
        sample_images = sample_images[:max_images].to(Config.DEVICE)
        sample_labels = sample_labels[:max_images].numpy()

        for idx, (image, true_label) in enumerate(zip(sample_images, sample_labels)):
            try:
                # Prepare figure for comparative visualization
                num_models = len(models_dict)
                fig, axes = plt.subplots(num_models, 3, figsize=(15, 5 * num_models), squeeze=False)

                image_np = image.permute(1, 2, 0).cpu().numpy() * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                image_np = np.clip(image_np, 0, 1)
                image_np = (image_np * 255).astype(np.uint8)

                row_idx = 0
                # Single models
                for model_name, model in models_dict.items():
                    try:
                        model.eval()
                        with torch.no_grad():
                            outputs = model(image.unsqueeze(0))
                            probabilities = torch.softmax(outputs, dim=1)
                            predicted_class = outputs.argmax(dim=1).item()
                            confidence = probabilities[0, predicted_class].item()
                            predicted_label = Config.CLASS_LABELS[predicted_class]

                        gradcam_img, gradcam_heatmap = xai_visualizer.grad_cam_plus_plus(model, image, predicted_class)
                        lrp_img, lrp_heatmap = xai_visualizer.lrp(model, image, predicted_class)

                        # Plot original image, Grad-CAM++, and LRP
                        axes[row_idx, 0].imshow(image_np)
                        axes[row_idx, 0].set_title(f'Original Image\nTrue: {Config.CLASS_LABELS[true_label]}\n{model_name}: {predicted_label} ({confidence:.2%})')
                        axes[row_idx, 0].axis('off')

                        axes[row_idx, 1].imshow(gradcam_img)
                        axes[row_idx, 1].set_title(f'{model_name} Grad-CAM++')
                        axes[row_idx, 1].axis('off')

                        axes[row_idx, 2].imshow(lrp_img)
                        axes[row_idx, 2].set_title(f'{model_name} Integrated Gradients')
                        axes[row_idx, 2].axis('off')

                        row_idx += 1

                    except Exception as e:
                        print(f"Error in comparative XAI for {model_name}, image {idx+1}: {e}")
                        continue

                plt.suptitle(f'Comparative XAI Visualizations - Image {idx+1}', fontsize=16)
                plt.tight_layout()
                save_path = f"{save_dir}/comparative_xai_image_{idx+1}.png"
                plt.savefig(save_path, dpi=300, bbox_inches='tight')
                plt.close()
                print(f"Comparative XAI visualization saved: {save_path}")

            except Exception as e:
                print(f"Error in comparative XAI for image {idx+1}: {e}")
                continue

# ---
# 12. FIXED XAI VISUALIZATIONS
# =============================================================================
# Purpose: Implement corrected Grad-CAM++ and LRP for visualizing important regions in images.

class XAIVisualizer:
    @staticmethod
    def grad_cam_plus_plus(model, image, target_class):
        """Implement Grad-CAM++ for visualizing important image regions."""
        model.eval()

        # Find the last convolutional layer
        target_layer = None
        for name, module in reversed(list(model.named_modules())):
            if isinstance(module, nn.Conv2d):
                target_layer = module
                break

        if target_layer is None:
            raise ValueError("No convolutional layer found in the model")

        # Hook to capture gradients and activations
        gradients = []
        activations = []

        def forward_hook(module, input, output):
            activations.append(output)

        def backward_hook(module, grad_input, grad_output):
            gradients.append(grad_output[0])

        # Register hooks
        forward_handle = target_layer.register_forward_hook(forward_hook)
        backward_handle = target_layer.register_backward_hook(backward_hook)

        try:
            # Forward pass
            image_tensor = image.unsqueeze(0).requires_grad_(True)
            output = model(image_tensor)

            # Backward pass
            model.zero_grad()
            score = output[0, target_class]
            score.backward()

            # Get gradients and activations
            if not gradients or not activations:
                raise ValueError("No gradients or activations captured")

            grad = gradients[0]  # Shape: (1, C, H, W)
            act = activations[0]  # Shape: (1, C, H, W)

            # Grad-CAM++ calculations
            alpha_num = grad.pow(2)
            alpha_denom = 2.0 * grad.pow(2) + (act * grad.pow(3)).sum(dim=(2, 3), keepdim=True)
            alpha_denom = torch.where(alpha_denom != 0, alpha_denom, torch.ones_like(alpha_denom))
            alpha = alpha_num / alpha_denom

            weights = (alpha * F.relu(grad)).sum(dim=(2, 3))  # Shape: (1, C)
            cam = (weights.unsqueeze(-1).unsqueeze(-1) * act).sum(dim=1, keepdim=True)  # Shape: (1, 1, H, W)
            cam = F.relu(cam)

            # Normalize
            cam = cam - cam.min()
            cam = cam / (cam.max() + 1e-8)

            # Resize to input image size
            cam = F.interpolate(cam, size=(224, 224), mode='bilinear', align_corners=False)
            cam = cam.squeeze().cpu().numpy()

            # Apply colormap
            cam = (cam * 255).astype(np.uint8)
            heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)

            # Convert original image for visualization
            image_np = image.permute(1, 2, 0).cpu().numpy()
            image_np = image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
            image_np = np.clip(image_np, 0, 1)
            image_np = (image_np * 255).astype(np.uint8)

            # Superimpose heatmap
            superimposed_img = cv2.addWeighted(image_np, 0.6, heatmap, 0.4, 0)

        finally:
            # Remove hooks
            forward_handle.remove()
            backward_handle.remove()

        return superimposed_img, heatmap

    @staticmethod
    def lrp(model, image, target_class):
        """Simplified LRP implementation using integrated gradients approach."""
        model.eval()

        # Create baseline (black image)
        baseline = torch.zeros_like(image)

        # Generate path from baseline to image
        num_steps = 50
        alphas = torch.linspace(0, 1, num_steps).to(image.device)

        gradients = []
        for alpha in alphas:
            interpolated = baseline + alpha * (image - baseline)
            interpolated = interpolated.unsqueeze(0).requires_grad_(True)

            output = model(interpolated)
            score = output[0, target_class]

            model.zero_grad()
            score.backward()

            gradients.append(interpolated.grad.clone())

        # Average gradients
        avg_gradients = torch.stack(gradients).mean(dim=0).squeeze(0)

        # Compute integrated gradients
        integrated_gradients = (image - baseline) * avg_gradients

        # Sum across color channels
        relevance = integrated_gradients.abs().sum(dim=0).cpu().numpy()

        # Normalize
        relevance = relevance / (relevance.max() + 1e-8)

        # Apply colormap
        relevance = (relevance * 255).astype(np.uint8)
        heatmap = cv2.applyColorMap(relevance, cv2.COLORMAP_JET)

        # Convert original image for visualization
        image_np = image.permute(1, 2, 0).cpu().numpy()
        image_np = image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        image_np = np.clip(image_np, 0, 1)
        image_np = (image_np * 255).astype(np.uint8)

        # Superimpose heatmap
        superimposed_img = cv2.addWeighted(image_np, 0.6, heatmap, 0.4, 0)

        return superimposed_img, heatmap


# ---
# 13. MAIN EXECUTION
# =============================================================================
# Purpose: Orchestrate the entire pipeline: data loading, model training, ensemble creation, evaluation, and visualization.

def main():
    print("Starting Fish Species Classification Pipeline...")
    print("="*70)
    setup_environment()

    # Load and balance data
    X, Y = DataManager.load_and_balance_data()
    train_loader, val_loader, test_loader, val_data, test_data = DataManager.create_data_loaders(X, Y)

    # Train individual models
    single_models = {}
    single_results = {}
    histories = {}
    evaluator = ModelEvaluator()
    visualizer = EnhancedVisualizations()

    for model_name in Config.MODELS:
        print(f"\nTraining {model_name}...")
        print("-"*50)

        # Optimize hyperparameters
        optimizer = ExpandedHyperparameterOptimizer(model_name, train_loader, val_loader)
        best_params = optimizer.optimize()

        # Create and train model
        model = ModelFactory.create_model(
            model_name,
            dropout_rate=best_params.get('dropout', 0.5),
            hidden_dim_multiplier=best_params.get('hidden_dim_multiplier', 0.5)
        )
        trainer = EnhancedModelTrainer(model, model_name, best_params)
        history = trainer.train(train_loader, val_loader)
        histories[model_name] = history
        single_models[model_name] = model

        # Evaluate model
        result = evaluator.evaluate_model(model, test_loader, model_name)
        single_results[model_name] = result

        # Plot ROC and confusion matrix
        visualizer.plot_roc_curves(result, model_name)
        visualizer.plot_confusion_matrix(result, model_name)

    # Plot training history for single models
    visualizer.plot_training_history(histories, model_type='single')

    # Create and evaluate ensembles
    ensemble_manager = EnsembleManager(single_models, val_data)
    ensemble_results, best_ensemble = ensemble_manager.test_ensemble_combinations()

    # Evaluate best ensemble on test set
    if best_ensemble:
        ensemble_name, ensemble_result = best_ensemble
        ensemble_models = ensemble_result['models']
        ensemble_method = ensemble_name.split('_')[-1]
        print(f"\nEvaluating best ensemble ({ensemble_name}) on test set...")

        test_dataset = FishDataset(test_data[0], test_data[1], DataManager.get_transforms(False))
        test_loader = DataLoader(test_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)

        if ensemble_method == 'learnable_weighted':
            ensemble_model = LearnableWeightedEnsemble(
                num_models=len(ensemble_models),
                num_classes=Config.NUM_CLASSES
            ).to(Config.DEVICE)
            ensemble_model.load_state_dict(
                torch.load(f"{Config.OUTPUT_DIR}/models/learnable_ensemble_{'+'.join(ensemble_models)}.pt")
            )
            ensemble_model.eval()

            all_preds = []
            all_probs = []
            all_labels = []
            total_loss = 0
            criterion = nn.CrossEntropyLoss()

            with torch.no_grad():
                for images, labels in test_loader:
                    images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
                    model_probs = []
                    for model_name in ensemble_models:
                        model = single_models[model_name]
                        outputs = model(images)
                        probs = torch.softmax(outputs, dim=1)
                        model_probs.append(probs)
                    model_probs = torch.stack(model_probs, dim=1)
                    outputs, _ = ensemble_model(model_probs)
                    loss = criterion(outputs, labels).item()
                    total_loss += loss
                    _, predicted = torch.max(outputs.data, 1)
                    all_preds.extend(predicted.cpu().numpy())
                    all_probs.extend(torch.softmax(outputs, dim=1).cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

            accuracy = accuracy_score(all_labels, all_preds)
            f1 = f1_score(all_labels, all_preds, average='macro')
            avg_loss = total_loss / len(test_loader)

            ensemble_results[ensemble_name]['test_accuracy'] = accuracy
            ensemble_results[ensemble_name]['test_f1'] = f1
            ensemble_results[ensemble_name]['test_loss'] = avg_loss
            ensemble_results[ensemble_name]['test_predictions'] = np.array(all_preds)
            ensemble_results[ensemble_name]['test_probabilities'] = np.array(all_probs)
            ensemble_results[ensemble_name]['test_true_labels'] = np.array(all_labels)

            print(f"Best Ensemble ({ensemble_name}) Test Results:")
            print(f"  Accuracy: {accuracy:.4f}")
            print(f"  F1 Score (Macro): {f1:.4f}")
            print(f"  Loss: {avg_loss:.4f}")

            visualizer.plot_roc_curves(ensemble_results[ensemble_name], ensemble_name)
            visualizer.plot_confusion_matrix(ensemble_results[ensemble_name], ensemble_name)

    # Plot model comparison
    visualizer.plot_model_comparison(single_results, ensemble_results)

    # Generate XAI visualizations
    visualizer.plot_gradcam_lrp(single_models, ensemble_results, test_loader, model_type='single')
    visualizer.plot_gradcam_lrp(single_models, ensemble_results, test_loader, model_type='ensemble')
    visualizer.plot_comparative_xai(single_models, ensemble_results, test_loader)

    # Save results
    results = {
        'single_results': {k: {kk: vv.tolist() if isinstance(vv, np.ndarray) else vv
                              for kk, vv in v.items()} for k, v in single_results.items()},
        'ensemble_results': {k: {kk: vv.tolist() if isinstance(vv, np.ndarray) else vv
                                for kk, vv in v.items()} for k, v in ensemble_results.items()},
        'best_ensemble': best_ensemble[0] if best_ensemble else None,
        'best_ensemble_f1': best_ensemble[1]['f1'] if best_ensemble else None
    }
    with open(f"{Config.OUTPUT_DIR}/results.json", 'w') as f:
        json.dump(results, f, indent=4)
    print(f"Results saved to {Config.OUTPUT_DIR}/results.json")

    # # REAL-WORLD TESTING (Commented out as requested)
    # # =============================================================================
    # # Purpose: Test the model on new, unseen images from a real-world dataset.
    # print("\nReal-World Testing (Commented Out)")
    # print("-"*50)
    # # real_world_images = load_real_world_data()  # Placeholder for loading real-world data
    # # real_world_predictions = []
    # # for image in real_world_images:
    # #     processed_image = preprocess_real_world_image(image)
    # #     with torch.no_grad():
    # #         output = best_model(processed_image.to(Config.DEVICE))
    # #         prediction = torch.argmax(output, dim=1).cpu().numpy()
    # #         real_world_predictions.append(prediction)
    # # save_real_world_results(real_world_predictions)

if __name__ == "__main__":
    main()

Starting Fish Species Classification Pipeline...
Using device: cuda
PyTorch version: 2.8.0+cu126
----------------------------------------------------------------------
Loading and preprocessing data...
Original data shape: (8407, 3, 224, 224)
Original class distribution: [3000 1185 2899  370  953]
Applying SMOTE for class balancing...


#End